In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/homework/sources/CC-Issues/sorted_dataset/
!du -a | cut -d/ -f2 | sort | uniq -c | sort -nr
# !ls

/content/drive/MyDrive/homework/sources/CC-Issues/sorted_dataset
  17338 trash_other
   3879 paper
   1519 glass
   1021 metal
    950 plastic_sm_lid
      7 misc_plastic
      1 221404	.


In [19]:
## CHECK DATASET SIZE
%cd /content/drive/MyDrive/homework/
!du -a | cut -d/ -f2 | sort | uniq -c | sort -nr

/content/drive/MyDrive/homework
  30396 six_category
   5076 check_images
   3126 sources
    371 unsorted
     16 logs
      1 .ipynb_checkpoints
      1 .DS_Store
      1 4355630	.


## Model

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import datetime
import PIL
from glob import glob
# %load_ext tensorboard

In [ ]:
model_name = "DataEngine-2"
model_root = "/content/drive/MyDrive/DataEngine/"
# TFLITE_FILE_PATH = model_root+model_name+'/'+model_name+".tflite"
SAVED_MODEL_PATH = model_root + model_name

data_dir = "/content/drive/MyDrive/homework/unsorted/"
img_height, img_width = 300, 300
batch_size = 16

In [ ]:
model = tf.keras.models.load_model(SAVED_MODEL_PATH)

In [ ]:
# data=tf.keras.preprocessing.image_dataset_from_directory(data_dir,label_mode=None, image_size=(img_height, img_width))

Found 19865 files belonging to 1 classes.


## Label

In [ ]:
paths = glob(os.path.join(data_dir, '*.jpg'))
print(len(paths), "unsorted images.")
print(paths[0])

In [ ]:
# Label Map
label_map = {0:'glass',
             1:'metal',
             2:'misc_plastic',
             3:'paper',
             4:'plastic_sm_lid',
             5:'trash_other'}
print(label_map)
sort_paths = {0:'/content/drive/MyDrive/homework/check_images/glass/',
              1:'/content/drive/MyDrive/homework/check_images/metal/',
              2:'/content/drive/MyDrive/homework/check_images/misc_plastic',
              3:'/content/drive/MyDrive/homework/check_images/paper/',
              4:'/content/drive/MyDrive/homework/check_images/plastic_sm_lid/',
              5:'/content/drive/MyDrive/homework/check_images/trash_other/'}
print(sort_paths)

{0: 'glass', 1: 'metal', 2: 'misc_plastic', 3: 'paper', 4: 'plastic_sm_lid', 5: 'trash_other'}
{0: '/content/drive/MyDrive/homework/check_images/glass/', 1: '/content/drive/MyDrive/homework/check_images/metal/', 2: '/content/drive/MyDrive/homework/check_images/misc_plastic', 3: '/content/drive/MyDrive/homework/check_images/paper/', 4: '/content/drive/MyDrive/homework/check_images/plastic_sm_lid/', 5: '/content/drive/MyDrive/homework/check_images/trash_other/'}


In [ ]:
paths = glob(os.path.join(data_dir, '*.jpg'))
print(len(paths), "unsorted images.")
for i in range(5000):
  # OPEN
  im_path = paths[i]
  img = PIL.Image.open(im_path)
  tens = tf.keras.preprocessing.image.img_to_array(img)
  tens = tf.expand_dims(tf.image.resize(tens, (img_height, img_width)), axis=0)

  if tens.shape == (1, 300, 300, 3):
    # PREDICT
    pred = model.predict(tens)
    pred_int = int(tf.math.argmax(pred, axis=1))
    pred_class = label_map[pred_int]
    print(i, im_path, pred, pred_int, pred_class)

    # MOVE
    mv_path = sort_paths[pred_int]
    !mv $im_path $mv_path



  # else:
  #   !rm paths[i]
  # target size (2000, 300, 300, 3)

In [ ]:
## TEST _ INFERENCE

# for i in range(len(paths)):
#   img = PIL.Image.open(paths[i])
#   tens = tf.keras.preprocessing.image.img_to_array(img)
#   tens = tf.expand_dims(tf.image.resize(tens, (img_height, img_width)), axis=0)
#   if tens.shape != (1, 300, 300, 3):
#     print(paths[i], tens.shape)
# !rm /content/drive/MyDrive/homework/unsorted/O_13535.jpg
# img = PIL.Image.open('/content/drive/MyDrive/homework/unsorted/O_13535.jpg')
# tens = tf.keras.preprocessing.image.img_to_array(img)

# tens = tf.expand_dims(tf.image.resize(tens, (img_height, img_width)), axis=0)
# print(tens.shape)

In [ ]:
### SPOT CHECK
### MOVE SINGLE IMAGE
path = '/content/drive/MyDrive/homework/check_images/trash_other/R_' + str(1600) + '.jpg'
print(path)
!mv $path /content/drive/MyDrive/homework/check_images/trash_other/checked

In [ ]:
### SPOT CHECK
### MOVE BATCH IMAGES
for i in range(11110,11111):
  path = '/content/drive/MyDrive/homework/check_images/trash_other/R_' + str(i) + '.jpg'
  print(path)
  !mv $path /content/drive/MyDrive/homework/check_images/paper

/content/drive/MyDrive/homework/check_images/trash_other/R_11110.jpg


In [ ]:
### MERGE DATASET TO SIX_CATEGORY
for category in label_map.values():
  if category == 'trash_other':
    mv_from = '/content/drive/MyDrive/homework/check_images/{}/checked/'.format(category)
  else:
    mv_from = '/content/drive/MyDrive/homework/check_images/{}/'.format(category)
  mv_to = '/content/drive/MyDrive/homework/six_category/{}/'.format(category)
  print(mv_from, mv_to)
  paths = glob(os.path.join(mv_from, '*.jpg'))
  for im in paths:
    !mv $im $mv_to

/content/drive/MyDrive/homework/check_images/glass/ /content/drive/MyDrive/homework/six_category/glass/
^C
/content/drive/MyDrive/homework/check_images/metal/ /content/drive/MyDrive/homework/six_category/metal/
/content/drive/MyDrive/homework/check_images/misc_plastic/ /content/drive/MyDrive/homework/six_category/misc_plastic/
/content/drive/MyDrive/homework/check_images/paper/ /content/drive/MyDrive/homework/six_category/paper/
/content/drive/MyDrive/homework/check_images/plastic_sm_lid/ /content/drive/MyDrive/homework/six_category/plastic_sm_lid/
/content/drive/MyDrive/homework/check_images/trash_other/checked/ /content/drive/MyDrive/homework/six_category/trash_other/


## Github Repo

In [ ]:
%cd /content/drive/MyDrive/Recycling-Data-Engine/
!git add E
!git config --global user.email "aqbewtra@gmail.com"
!git config --global user.name "Avi Bewtra"
# !git commit -m ""

/content/drive/MyDrive/Recycling-Data-Engine
[master c63c4fa] Create README.md
 1 file changed, 7 insertions(+)
 create mode 100644 README.md


In [ ]:
!git push origin master --force

Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 600 bytes | 300.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/aqbewtra/Recycling-Data-Engine.git
   a3e1626..c63c4fa  master -> master
